# Avant de commencer

Créer une variable `u_0` correspondant à la valeur qui vous a été attribuée.

In [1]:
let u0_test = 29818;;

val u0_test : int = 29818


-------------------
Une entreprise de $n$ employés ($n \geqslant 2$) veut récompenser $k$ ses meilleurs vendeurs. Chaque employé, désigné par son numéro de badge $i$ (où $ 0 \leqslant i < n$), a réalisé un chiffre de ventes $a_i$ au cours de  l'année 2020. On suppose que pour tout entier $i$, $a_i$ est un entier.

L'objectif est de déterminer le $k$-ième chiffre de ventes le plus élevé, ce qui correspondra au seuil à partir duquel les employés auront le droit à une prime.


**Dans tout le sujet, on pourra utiliser ` Array.make` et ` Array.length`, mais pas d'autres fonctions du module ` Array`**

# Préliminaires

On considère une suite d'entiers $(u_k)_{k \in \mathbb{N}}$ vérifiant la relation de récurrence :
$$\forall k \in \mathbb{N}, u_{k+1} = (15091\times u_k)\text{ mod } 64007$$

#### Question 1
Écrire une fonction `suivant : int -> int` qui, lorsqu'elle prend en argument l'entier correspondant à $u_k$, renvoie $u_{k+1}$.

In [2]:
let suivant uk =
  15091*uk mod 64007
;;

val suivant : int -> int = <fun>


#### Question 2
Écrire une fonction `tab_u : int -> int -> int array` qui prend en argument la valeur de $u_0$ et un entier $n$ et qui renvoie le tableau $[|u_0;u_1;\dots;u_{n-1}|]$ des $n$ premières valeurs de la suite $u$.

In [3]:
let tab_u u0 n =
  let tab = Array.make n u0 in
  for i = 1 to n-1 do
    tab.(i) <- suivant tab.(i-1)
  done;
  tab
;;

val tab_u : int -> int -> int array = <fun>


#### Question 3
Calculer les valeurs de $u_{20}$, $u_{2000}$, $u_{20000}$.

In [4]:
let u_test n = let tab_test = tab_u u0_test (n+1) in tab_test.(n);;

let u_20 = u_test 20  ;;
let u_2000 = u_test 2000 ;;
let u_20000 =  u_test 20000;;

val u_test : int -> int = <fun>


val u_20 : int = 12660


val u_2000 : int = 29816


val u_20000 : int = 53256


Dans toute la suite, on considère que le tableau des $n$ chiffres de ventes ($n$ pouvant dépendre de la question) est donné par ` tab_u u0 n` où ` u0` est votre valeur pour $u_0$.

# Partie I

#### Question 4
Dans cette question, on suppose que $k = 1$. 
-  Écrire une fonction ` meilleure_vente : 'a array -> 'a` qui prend en argument le tableau des chiffres de ventes et qui renvoie le meilleur chiffre de ventes.

In [5]:
let meilleure_vente tab =
  let n = Array.length tab in
  let maxi = ref tab.(0) in
  for i = 1 to n-1 do
    if tab.(i) > !maxi
    then maxi := tab.(i)
  done;
  !maxi
;;

val meilleure_vente : 'a array -> 'a = <fun>


- Calculer la valeur de la meilleure vente pour $n = 1000$.

In [6]:
let meilleure_vente_1000 = meilleure_vente (tab_u u0_test 1000);;

val meilleure_vente_1000 : int = 63984


#### Question 5
Dans cette question, on suppose que la distribution des chiffres de ventes est relativement uniforme. Une approximation du seuil de ventes à atteindre pour obtenir la prime est alors donnée par la moyenne du chiffre de ventes minimum et du chiffre de ventes maximum, respectivement pondérés par $(k-1)$ et $n-k$.

- Écrire une fonction ` approx_seuil: int array -> int -> int`  prenant en argument le tableau des chiffres de ventes et l'entier $k$ et renvoyant cette moyenne pondérée arrondie à l'entier inférieur. 

In [7]:
let approx_seuil tab k =
  let n = Array.length tab in
  let maxi = ref tab.(0) in
  let mini = ref tab.(0) in
  for i = 1 to n-1 do
    if tab.(i) > !maxi
    then maxi := tab.(i)
    else
      if tab.(i) < !mini
       then mini:= tab.(i)    
  done;
  ((k-1) * !mini + (n-k) * !maxi)/(n-1)
  (* La somme des deux coefficients vaut (n-1)*)
;;

val approx_seuil : int array -> int -> int = <fun>


- Calculer le seuil obtenu avec la fonction précédente pour $k = 200$ et $n=5000$.

In [8]:
let seuil_approx_200_5000 = approx_seuil (tab_u u0_test 5000) 200;;

val seuil_approx_200_5000 : int = 61455


# Partie II
On traite maintenant le cas général, en ne faisant aucune hypothèse sur la répartition des ventes.

Pour calculer le $k$-ième meilleur chiffre de ventes, on va trier partiellement le tableau dans l'ordre décroissant, de sorte que les $k$ plus grands éléments soient à leur place dans le tableau.

#### Question 6
Écrire une fonction `echange : 'a array -> int -> int -> unit` qui prend en argument un tableau et deux entiers $i$ et $j$ et échangeant en place les éléments d'indices $i$ et $j$ du tableau. Aucune vérification sur $i$ et $j$ n'est demandée.

In [9]:
let echange tab i j =
  let tmp = tab.(i) in
  tab.(i) <- tab.(j);
  tab.(j) <- tmp
;;

val echange : 'a array -> int -> int -> unit = <fun>


#### Question 7 
Écrire une fonction ` seuil : 'a array -> int -> 'a` prenant en argument le tableau des chiffres de ventes et l'entier $k$ et renvoyant le $k$-ième plus grand élément du tableau.

In [10]:
let seuil tab k =
  let n = Array.length tab in
  for i = 0 to k-1 do
    let imax = ref i in
    for j = i+1 to n-1 do
      if tab.(j) > tab.(!imax)
      then imax := j
    done;
    echange tab !imax i
  done;
  tab.(k-1)
;;

val seuil : 'a array -> int -> 'a = <fun>


#### Question 8
Calculer le seuil obtenu avec la fonction précédente pour $k = 200$ et $n=5000$.

In [11]:
let seuil_II_200_5000 = seuil (tab_u u0_test 5000) 200;;

val seuil_II_200_5000 : int = 61792


# Partie III
Le nombre d'employés $n$ étant particulièrement grand, on cherche à optimiser le calcul du $k$-ième meilleur chiffre de ventes.

Pour cela, on procède comme suit : on choisit un employé $i$, et on place son chiffre de ventes $v = a_i$ à l'indice $l$ qu'il aurait dans le tableau si celui-ci était trié dans l'ordre décroissant, en réordonnant le tableau de sorte que $a_j > v$ si $j < l$ et $a_j \leqslant v$ et $j>l$. Si l'indice $l$ correspond au $k$-ième élément du tableau, on a terminé, sinon, on recommence avec la partie à gauche ou à droite de $l$ dans le tableau.

#### Question 9
Soient $g$ et $d$ deux entiers tels que $0 \leqslant g < d < n$ et soit $v = a_g$. Écrire une fonction ` partition : 'a array -> int -> int -> int` qui prend en argument le tableau des chiffres de ventes et les entiers $g$ et $d$ (dans cet ordre), qui réordonne le tableau en place entre les indices $g$ et $d$ compris de sorte que $a_j > v$ pour $g \leqslant j < l$ et $a_j \leqslant v$ pour $l < j \leqslant d$, et qui renvoie l'indice $l$ de la valeur $v$ après modification du tableau.

*Indication :* On procédera par des échanges ; on pourra laisser $v$ à l'indice $g$ jusqu'à avoir fini de parcourir la portion de tableau puis procéder à un dernier échange avec l'indice $l$.

In [12]:
let partition tab g d =
  let v = tab.(g) in
  let l = ref g in
  for k = g+1 to d do
    if tab.(k) > v
    then (incr l ; echange tab k !l)
  done;
  echange tab g !l;
  !l   
;;

val partition : 'a array -> int -> int -> int = <fun>


#### Question 10
Calculer l'indice obtenu avec la fonction précédente pour $n = 5000$, $g = 1000$ et $d = 2000$.

In [13]:
let indice_5000_1000_2000 = partition (tab_u u0_test 5000) 1000 2000;;

val indice_5000_1000_2000 : int = 1926


#### Question 11
Écrire une fonction récursive ` seuil_rec : 'a array -> int -> int -> int -> 'a` qui prend en argument le tableau des chiffres de ventes, trois entiers $j$, $g$ et $d$ (dans cet ordre) tels que $0 \leqslant g \leqslant j \leqslant d$ et qui renvoie l'élément qui serait à l'indice $j$ dans si la portion du tableau entre les indices $g$ et $d$ (inclus) était triée dans l'ordre décroissant. On pourra utiliser la fonction ` partition`.

In [14]:
let rec seuil_rec tab j g d =
  let l = partition tab g d in
  if l = j
  then tab.(l)
  else
    if l > j
    then seuil_rec tab j g (l-1)
    else seuil_rec tab j (l+1) d
;;

val seuil_rec : 'a array -> int -> int -> int -> 'a = <fun>


#### Question 12
En déduire une fonction `seuil_bis : 'a array -> int -> 'a` prenant en argument le tableau des chiffres de ventes et l'entier $k$ et renvoyant le $k$-ième plus grand élément du tableau.

In [15]:
let seuil_bis tab k =
  seuil_rec tab (k-1) 0 (Array.length tab - 1)
;;

val seuil_bis : 'a array -> int -> 'a = <fun>


#### Question 13
Calculer le seuil obtenu avec la fonction précédente :
+ pour $k = 1000$ et $n=20000$ ;
+ pour $k = 10000$ et $n=500000$.

In [16]:
let seuil_III_1000_20000 = seuil_bis (tab_u u0_test 20000) 1000;;
let seuil_III_10000_500000 = seuil_bis (tab_u u0_test 500000) 10000;;
(*BEGIN SOLUTION*)
(*END SOLUTION*)

val seuil_III_1000_20000 : int = 60826


val seuil_III_10000_500000 : int = 62725
